# Technical Exercise – Data Scientist

Tasks:
1.	Which factors best predict anti-social behaviour? -> multivariate regression problem -> Feature Selection -> Random Forest Regression,Linear Regression (PyCarret AutoML) <br><br>
2.	Data were missing for the St. John’s ward in Woking due to a recording error. Given what we know about the demographics of this area, can you predict the level of anti-social behaviour it might experience? -> Imputation Techniques, Domain Knowledge & Similar Ward Analysis <br><br>
3.	A councillor has raised specific concerns about the Walton area of Elmbridge. Given recent trends, can you predict whether this area is of notable concern for the coming months? -> Time Series Analysis (SKtime AutoML) <br><br>
4.	Briefly, what actions might you recommend based on all the above? Report



In [2]:
## python 3.11
# !pip install -q pycaret
# !pip install -q sktime

In [3]:
import pandas as pd
from IPython.display import display

## Data

In [5]:
df_indecators = pd.read_csv('data/indicators.csv')
df_geo = pd.read_excel(
  'data/LSOA 2021 to Ward 2022 to LA 2022 lookup England and Wales.xlsx')
df_street_crime = pd.read_excel('data/Surrey_street_crime.xlsx')
print('indecators')
display(df_indecators.head())
print('geos')
display(df_geo.head())
print('street crime')
display(df_street_crime.head())

indecators


,Area Type,Area Name,Unemployment benefit (JSA and Universal Credit) (Sep 2023),Disability benefit (Feb 2023),Pensioners in poverty (Pension Credit),Youth unemployment (18-24 receiving JSA or Universal Credit) (Sep 2023),Percentage of children in poverty (after housing costs),Total population (2020),Population aged 0-15 (2020),Population of working age (16-64) (2020),...,Children 0-19 in relative low-income out of work families (as % of all in relative low income families) (2021/2022),Children 0-19 in relative low-income in-work families (as % of all in relative low income families) (2021/2022),Children aged 0-19 in relative low-income families (2021/2022),Households with no car (Census 2021),Broadband speed (July 2020),Premises with broadband speeds below the Universal Service Obligation (USO) (2019),"Greenspace coverage, public parks and gardens (2017)","Greenspace coverage, total (2017)","Voter Turnout at Local Elections (2016-2019, depending on when the Local Council held their most recent election)",Households in Fuel Poverty (2021)
0,County,Surrey,2.158800,1.551332,6.090448,2.654761,19.637423,1199872,19.745940,61.093683,...,16.225847,83.759459,9.341381,12.714035,78.051656,0.725177,1.767591,6.179592,35.315675,7.347314
1,Elmbridge Wards,Claygate,1.977967,0.998146,5.183727,0.000000,16.671953,7013,21.317553,56.951376,...,14.606742,NaN,4.944444,9.356101,92.168959,0.000000,0.003211,3.814245,41.556777,7.654666
2,Elmbridge Wards,Cobham and Downside,1.983114,1.503926,7.779419,2.296451,32.096514,9043,21.220834,56.319805,...,15.311005,NaN,9.067245,12.506724,79.493225,0.517879,2.373987,3.116821,30.212631,9.071674
3,Elmbridge Wards,Esher,2.075204,1.307190,5.772402,2.622951,24.072676,9486,21.389416,59.434957,...,15.950920,NaN,6.546185,9.899610,85.752733,0.406814,2.266658,9.905930,37.557746,8.028017
4,Elmbridge Wards,Hersham Village,2.318959,1.457346,9.154930,4.473684,20.462943,8440,24.928910,58.246446,...,NaN,NaN,7.419758,12.428455,95.038102,0.000000,0.878644,15.083548,35.337034,7.075472


geos


,LSOA21CD,LSOA21NM,LSOA21NMW,WD22CD,WD22NM,WD22NMW,LAD22CD,LAD22NM,LAD22NMW,ObjectId
0,E01004766,Bolton 005A,NaN,E05000650,Astley Bridge,NaN,E08000001,Bolton,NaN,1
1,E01005347,Oldham 028E,NaN,E05000722,Chadderton South,NaN,E08000004,Oldham,NaN,2
2,E01004890,Bolton 004E,NaN,E05000661,Horwich North East,NaN,E08000001,Bolton,NaN,3
3,E01004770,Bolton 003B,NaN,E05000650,Astley Bridge,NaN,E08000001,Bolton,NaN,4
4,E01004888,Bolton 004C,NaN,E05000661,Horwich North East,NaN,E08000001,Bolton,NaN,5


street crime


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Borough name,Crime type,Last outcome category,Context
0,8720eda20c741d07c0517f413ac9f87bdc7eceaa585f1b...,2020-10-01,Surrey Police,Surrey Police,0.107482,51.427084,On or near St John'S Road,E01000459,Bexley 027F,Bexley,Violence and sexual offences,Unable to prosecute suspect,NaN
1,NaN,2020-10-01,Surrey Police,Surrey Police,-0.774968,51.332407,On or near Station Approach,E01016196,Bracknell Forest 015B,Bracknell Forest,Anti-social behaviour,NaN,NaN
2,b78b8b4c93252c02f252b8255ed49e4cf240ffe1a0b568...,2020-10-01,Surrey Police,Surrey Police,-0.774968,51.332407,On or near Station Approach,E01016196,Bracknell Forest 015B,Bracknell Forest,Drugs,Offender given a caution,NaN
3,b8d36a8811b637fbe1191740294f9b2d60684a309dcdcb...,2020-10-01,Surrey Police,Surrey Police,0.021878,51.322459,On or near Oaklands Lane,E01000656,Bromley 039C,Bromley,Criminal damage and arson,Status update unavailable,NaN
4,0134df90eabcc147e9a760c178194a6f03c12147d675db...,2020-10-01,Surrey Police,Surrey Police,0.021878,51.322459,On or near Oaklands Lane,E01000656,Bromley 039C,Bromley,Criminal damage and arson,Status update unavailable,NaN


**Info**
* indicators: including features index 0 surry county info, rest all other wards
* geos: lookup table
* street_crime target data<br><br>

### Statitstical analysis

* How does income level relate to educational attainment? (Correlation Analysis)
* Are there significant differences in unemployment rates between different ethnic groups? (Hypothesis Testing)
* Are there areas with a higher concentration of street crimes? (Spatial Analysis)